In [16]:
from langchain_community.document_loaders import PyPDFLoader
import pprint

from sentence_transformers import SentenceTransformer


In [17]:
file_path = (
    "/Volumes/mashenka/delete/layout-parser-paper.pdf"
)

loader = PyPDFLoader(file_path)
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [22]:


# Load embedding model
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Example document
text = "ChromaDB is a vector database for AI applications."

# Generate embeddings
embedding = embedding_model.encode(text)

print(embedding.shape)  # Output: (384,)


('LayoutParser: A Uniﬁed Toolkit for Deep\n'
 'Learning Based Document Image Analysis\n'
 'Zejiang Shen1 (\x00 ), Ruochen Zhang2, Melissa Dell3, Benjamin Charles '
 'Germain\n'
 'Lee4, Jacob Carlson3, and Weining Li5\n'
 '1 Allen Institute for AI\n'
 'shannons@allenai.org\n'
 '2 Brown University\n'
 'ruochen zhang@brown.edu\n'
 '3 Harvard University\n'
 '{melissadell,jacob carlson}@fas.harvard.edu\n'
 '4 University of Washington\n'
 'bcgl@cs.washington.edu\n'
 '5 University of Waterloo\n'
 'w422li@uwaterloo.ca\n'
 'Abstract. Recent advances in document image analysis (DIA) have been\n'
 'primarily driven by the application of neural networks. Ideally, research\n'
 'outcomes could be easily deployed in production and extended for further\n'
 'investigation. However, various factors like loosely organized codebases\n'
 'and sophisticated model conﬁgurations complicate the easy reuse of im-\n'
 'portant innovations by a wide audience. Though there have been on-going\n'
 'eﬀorts to improve

In [12]:
import fitz
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from PIL import Image


def plot_pdf_with_boxes(pdf_page, segments):
    pix = pdf_page.get_pixmap()
    pil_image = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

    fig, ax = plt.subplots(1, figsize=(10, 10))
    ax.imshow(pil_image)
    categories = set()
    category_to_color = {
        "Title": "orchid",
        "Image": "forestgreen",
        "Table": "tomato",
    }
    for segment in segments:
        points = segment["coordinates"]["points"]
        layout_width = segment["coordinates"]["layout_width"]
        layout_height = segment["coordinates"]["layout_height"]
        scaled_points = [
            (x * pix.width / layout_width, y * pix.height / layout_height)
            for x, y in points
        ]
        box_color = category_to_color.get(segment["category"], "deepskyblue")
        categories.add(segment["category"])
        rect = patches.Polygon(
            scaled_points, linewidth=1, edgecolor=box_color, facecolor="none"
        )
        ax.add_patch(rect)

    # Make legend
    legend_handles = [patches.Patch(color="deepskyblue", label="Text")]
    for category in ["Title", "Image", "Table"]:
        if category in categories:
            legend_handles.append(
                patches.Patch(color=category_to_color[category], label=category)
            )
    ax.axis("off")
    ax.legend(handles=legend_handles, loc="upper right")
    plt.tight_layout()
    plt.show()


def render_page(doc_list: list, page_number: int, print_text=True) -> None:
    pdf_page = fitz.open(file_path).load_page(page_number - 1)
    page_docs = [
        doc for doc in doc_list if doc.metadata.get("page_number") == page_number
    ]
    segments = [doc.metadata for doc in page_docs]
    plot_pdf_with_boxes(pdf_page, segments)
    if print_text:
        for doc in page_docs:
            print(f"{doc.page_content}\n")

In [13]:
render_page(docs, 5)

NameError: name 'docs' is not defined